# Notebook description
<!---define operatorname var-->
$\DeclareMathOperator{\Var}{Var}$
<!---define operatorname var-->This notebook compares Poisson vs uniform spike train encoding. We use the SNR and the latency as specifications.

Specifications:
 - $SNR=20$ or equivalently $NSR=.05$ or that we have a $5\%$ error.
 - Latency = $.01$s = $10$ms

<div style="display:block; border: 1px solid black; padding: 15px;background-color: #EEEEFF;margin: 10px">
<h2>Filtered Poisson spike train statistics</h2>

\begin{align}
E[X] &= \lambda \\
\operatorname{Var}(X) &= \frac{\lambda}{2\tau} \\
SNR &= \sqrt{2\lambda\tau} \\
\end{align}
</div>

<div style="display:block; border: 1px solid black; padding: 15px;background-color: #EEEEFF;margin: 10px">
<h2>Filtered uniform spike train statistics</h2>

$$
E[x] = \lambda
$$

<br>

\begin{align*}
\operatorname{Var}(x) &= \lambda^2\left(\frac{1}{2\lambda\tau}\coth\left(\frac{1}{2\lambda\tau}\right)-1\right) \\
\operatorname{Var}(x) &\approx \frac{1}{12\tau^2} & \text{at high }\lambda\tau \\
\end{align*}

<br>

\begin{align*}
SNR &= \frac{1}{\sqrt{\frac{1}{2\lambda\tau}\coth\left(\frac{1}{2\lambda\tau}\right)-1}} \\
SNR &\approx \sqrt{12}(\lambda\tau) & \text{at high }\lambda\tau\\
\end{align*}

</div>

In [1]:
import numpy as np
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
def u_snr(lam, tau):
#     return 1./np.sqrt(1./(2.*lam*tau)/np.tan(1./(2.*lam*tau))-1)
    return 1./np.sqrt(1./(2.*lam*tau)*(1.+np.exp(-1./lam/tau))/(1.-np.exp(-1./lam/tau))-1)

def get_u_lam(snr, tau, approx=False):
    if approx:
        lam = snr / np.sqrt(12) / tau
    else:
        ret = fsolve(lambda x: u_snr(x, tau) - snr, 1./tau)
        lam = ret[0]
        assert np.isclose(u_snr(lam, tau), snr), '%f' % lam
    return lam

def get_p_lam(snr, tau):
    lam = snr**2/2./tau
    return lam

First let's check the required Poisson and uniform spike rates given the specified SNR and latency. That is, if we simply encoded a value using a Poisson or uniform spike train, what rate would we need to spike at to meet the SNR and latency spec? This is equivalent to assuming we're using a single neuron to encode our value.

In [14]:
snr = 20.
tau = .01

lam_p = get_p_lam(snr, tau)
lam_u_approx = get_u_lam(snr, tau, approx=True)
lam_u = get_u_lam(snr, tau)

print 'Poisson spike encoding requires %.0f spks/s' % lam_p
print 'Uniform spike encoding requires %.0f spks/s (approx)' % lam_u_approx
print 'Uniform spike encoding requires %.0f spks/s' % lam_u

Poisson spike encoding requires 20000 spks/s
Uniform spike encoding requires 577 spks/s (approx)
Uniform spike encoding requires 577 spks/s


Note that applying a decode weight $d$ to a single neuron does not change the $SNR$ because $E[dX]=dE[X]$ and $\Var(dX)=d^2\Var(X)$ so $SNR(dX) = \frac{E[dX]}{\sqrt{\Var(dX)}} = \frac{dE[X]}{d\sqrt{\Var(X)}}=SNR(X)$

In practice, we'll never use a single neuron because we need a diversity of tuning curves to approximate arbitrary functions other. Let's say we use $N$ neurons and take the sum of their spike trains as encoding the signal.

Now, 

$$
X = X_1 + X_2 + \ldots + X_N
$$